In [ ]:
#importing required packages
import numpy as np
import matplotlib.image as img
import matplotlib.pyplot as plt
import glob  
import datetime as dt
from sklearn import svm, metrics

In [ ]:
#loading train image data
train_images = []
for filename in glob.glob("/path to your folder/*.png"):
    cur_image = img.imread(filename)
    train_images.append(cur_image)

In [ ]:
#plotting some images 
for i in range(5):
    plot = plt.imshow(train_images[i])
    plt.show()

In [ ]:
#converting the list into a numpy array
train_images = np.asarray(train_images)

#converting training images array into 2-D array
train_images = train_images[:,:,:,0]
n_sample, x, y = train_images.shape
train_images = train_images.reshape(n_sample, x*y)

#loading the  targets for the images
train_targets = np.genfromtxt("/path to your folder/Train.csv", delimiter = ",")

#Exploring csv file of labels
print("CSV file of the labels :\n " , train_targets , "\n")
print("Shape of the file : \n" , train_targets.shape, "\n")

#getting only labels
train_targets = train_targets[1:len(train_targets), 1]

In [ ]:
#applying grid search, trying to find the best model 

#importing GridSearchCV package
from sklearn.model_selection import GridSearchCV

#setting parameters and their ranges(taking known best parameters due to limited computation power)
gamma_range = np.array([0.05, 0.5])
C_range = np.array([1, 5])
parameters = {'kernel' : ['rbf'], 'C' : C_range, 'gamma' : gamma_range }

#creating the classifiers
svm_classifier = svm.SVC()
grid_classifier = GridSearchCV(estimator = svm_classifier, param_grid = parameters, n_jobs = 1, verbose = 2)

#training the models and calculating time for the training
start_time = dt.datetime.now()

print('Training for the models starts now : {}'.format(str(start_time)), '\n')

grid_classifier.fit(train_images, train_targets)

run_time = dt.datetime.now() - start_time

print('Time taken to train the models : {}'.format(str(run_time)), '\n')

#collecting the best parameters
classifier = grid_classifier.best_estimator_
params = grid_classifier.best_params_

#showing the scores of all the models
scores = grid_classifier.cv_results_['mean_test_score'].reshape(len(C_range), len(gamma_range))
print("Scores of all the models : \n", scores.ravel(), '\n')

In [ ]:
#now, predicting the test data

#loading test image data
test_images = []
for filename in glob.glob("/path to your folder/*.png"):
    cur_image = img.imread(filename)
    test_images.append(cur_image)
    
#converting the list into a numpy array
test_images = np.asarray(test_images)

#converting test images array into 2-D array
test_images = test_images[:,:,:,0]
n_sample, x, y = test_images.shape
test_images = test_images.reshape(n_sample, x*y)

#predicting the targets
predicted_targets = classifier.predict(test_images)

In [ ]:
#saving results in a .csv file
np.savetxt("prediction.csv", predicted_targets, delimiter = ',')